# This will setup the baseline experiment for the neural network. And subsequent improvements will be made on this network


This part sets up the dataset

In [1]:
STUDENTID = 567     # this will be used for random states

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import logging
from utils import *
import torch.optim as optim
import os
import shutil
import os

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the data
])

# define the label transformations, from int64 to float32
transform_label = transforms.Compose([
  #transforms.ToTensor()
])

# Download and load CIFAR-100 datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True, target_transform=transform_label)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True, target_transform=transform_label)


# Calculate the sizes for train, validation, and test sets
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

# Split the train dataset into train and validation sets using a random seed
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [train_size, valid_size], generator=torch.Generator().manual_seed(STUDENTID))

# Create data loaders for training, validation, and test sets
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(valid_dataset))
print("Number of test examples:", len(test_dataset))



Files already downloaded and verified
Files already downloaded and verified
Number of training examples: 40000
Number of validation examples: 10000
Number of test examples: 10000


Setting up the train, evaluation, and test functions


Setting up the neural network model


In [3]:
# creating a model that automatically runs the forward function i guess, since it is easier
import torch
import torch.nn as nn
import torch.optim as optim
from utils import *
from customModels import *

In [4]:
model = ProjectNN2()
modelName = 'ProjectNN2'

argDict = {
  'lr': 0.001,
  'maxEpoch': 250,
  'idleEpoch': 25,
  'outputName': modelName,
  'optimizer': optim.SGD(model.parameters(), lr=0.001),
  'criterion': nn.CrossEntropyLoss()
}

# setting up the logger
loggerName = modelName + '.log'
loggerName = os.path.join(argDict['outputName'], loggerName)
logger = MyLogger(loggerName)
argDict['logger'] = logger

# just to initilalize the files
logger.log('training starts here')

# training and saving model to dictionary
outputDict = train(model, argDict, train_loader, val_loader, test_loader)

# loading the best model, and then sending it off to testing
model = load_model_from_file(model, argDict['outputName'], argDict['outputName'])

test_accuracy = test(model, argDict, test_loader)
tempString = 'testing accuracy of ' + argDict['outputName'] + " is: " + str(test_accuracy)
logger.log(tempString)

argDict['test_accuracy'] = str(test_accuracy)
save_dict_to_file(outputDict, argDict['outputName'], argDict['outputName'])

del model
del argDict

# Define the folder you want to zip and the output zip file name
folder_to_zip = modelName
output_zip_file = modelName + ".zip"

# Use shutil.make_archive to create the zip file
shutil.make_archive(output_zip_file, 'zip', folder_to_zip)

os.rename(output_zip_file + '.zip', output_zip_file)

training starts here
currently at epoch 0 train accuracy: tensor(0.1857, device='cuda:0') loss of: 2.17796244430542 eval accuracy: tensor(0.2502, device='cuda:0')
currently at epoch 1 train accuracy: tensor(0.2730, device='cuda:0') loss of: 1.9656897855758666 eval accuracy: tensor(0.2994, device='cuda:0')
currently at epoch 2 train accuracy: tensor(0.3208, device='cuda:0') loss of: 1.8487550987243653 eval accuracy: tensor(0.3491, device='cuda:0')
currently at epoch 3 train accuracy: tensor(0.3570, device='cuda:0') loss of: 1.7678899890899658 eval accuracy: tensor(0.3657, device='cuda:0')
currently at epoch 4 train accuracy: tensor(0.3796, device='cuda:0') loss of: 1.709186304283142 eval accuracy: tensor(0.3973, device='cuda:0')
currently at epoch 5 train accuracy: tensor(0.3999, device='cuda:0') loss of: 1.6616207738876343 eval accuracy: tensor(0.4084, device='cuda:0')
currently at epoch 6 train accuracy: tensor(0.4119, device='cuda:0') loss of: 1.6243211349487305 eval accuracy: tensor